In [ ]:
run_id = None 
pipeline_name = None
table_name = None
source_snapshot_ts = None
rows_in = None
rows_out = None
status = "Failed"

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 3, Finished, Available, Finished)

In [ ]:
from datetime import datetime
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType, DoubleType
import uuid

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 4, Finished, Available, Finished)

In [ ]:
now = datetime.now()

source_snapshot_ts = None
rows_in = None
rows_out = None

try:
    source_snapshot_ts = datetime.strptime(source_snapshot_str, "%Y-%m-%d")
except:
    pass

try:
    rows_in = int(rows_in_str)
except:
    pass

try:
    rows_out = int(rows_out_str)
except:
    pass





StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 6, Finished, Available, Finished)

In [ ]:
freshness_hours = None
freshness_sla_ok = None
completeness_ratio = None
completeness_sla_ok = None
sla_met = None


if status.lower() == "succeeded":
    try:
        if source_snapshot_ts:
            freshness_hours = (now - source_snapshot_ts).total_seconds() / 3600
            print(freshness_hours)
            freshness_sla_ok = freshness_hours <= 12

        if rows_in is not None and rows_out is not None and rows_in > 0:
            completeness_ratio = rows_out / rows_in
            completeness_sla_ok = completeness_ratio >= 0.98

        if freshness_sla_ok is not None and completeness_sla_ok is not None:
            sla_met = freshness_sla_ok and completeness_sla_ok
            
    except Exception as e:
        status = "Failed"

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 7, Finished, Available, Finished)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType, DoubleType

log_schema = StructType([
    StructField("run_id", StringType(), False),
    StructField("pipeline_name", StringType(), False),
    StructField("table_name", StringType(), False),
    StructField("source_snapshot_ts", TimestampType(), True),
    StructField("run_ts", TimestampType(), False),
    StructField("rows_in", IntegerType(), True),
    StructField("rows_out", IntegerType(), True),
    StructField("freshness_hours", DoubleType(), True),
    StructField("completeness_ratio", DoubleType(), True),
    StructField("status", StringType(), True),
    StructField("sla_met", BooleanType(), True)
])



log_data = [(
    run_id,
    pipeline_name,
    table_name,
    source_snapshot_ts,
    now,
    rows_in,
    rows_out,
    freshness_hours,
    completeness_ratio,
    status,
    sla_met
)]


df_log = spark.createDataFrame(log_data, schema=log_schema)

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 8, Finished, Available, Finished)

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS monitoring_etl_run_log (
    run_id STRING,
    pipeline_name STRING,
    table_name STRING,
    source_snapshot_ts TIMESTAMP,
    run_ts TIMESTAMP,
    rows_in INT,
    rows_out INT,
    freshness_hours DOUBLE,
    completeness_ratio DOUBLE,
    status STRING,
    sla_met BOOLEAN
)
USING DELTA
""")

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 9, Finished, Available, Finished)

DataFrame[]

In [ ]:
df_log.write.mode("append").saveAsTable("monitoring_etl_run_log")

StatementMeta(, f4f163ac-b570-4ff9-a929-135b13ead2cd, 10, Finished, Available, Finished)